In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.append('/home/josedvq/furnace')
sys.path.append('/home/josedvq/furnace/slowfast')
import logging
import pickle
import traceback

In [3]:
import torch

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, GroupKFold
from IPython.display import clear_output
import pytorch_lightning as pl
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)
# set the cudnn
torch.backends.cudnn.benchmark=False
torch.backends.cudnn.deterministic=True

/home/josedvq/.virtualenvs/lared/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from lared_dataset.data_loading.dataset import FatherDataset, FatherDatasetSubset, RemoveConfidence
from lared_dataset.data_loading.extractors import VideoExtractor, AccelExtractor
from lared_dataset.baselines.video.models import make_slow_pretrained_body as get_video_feature_extractor
from lared_dataset.baselines.video.transforms import get_kinetics_val_transform
from lared_dataset.constants import (
    processed_data_path, 
    processed_accel_path,
    processed_videos_path,
    examples_path)
from lared_dataset.baselines.train import System, train, test

/home/josedvq/.virtualenvs/lared/lib/python3.8/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/josedvq/.virtualenvs/lared/lib/python3.8/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [5]:
def do_cross_validation(do_train, ds, input_modalities, seed, prefix=None, deterministic=False):
    
    cv_splits = list(GroupKFold(n_splits=3).split(range(len(ds)), groups=ds.get_groups()))
    
    all_results = []
    for f, (train_idx, test_idx) in enumerate(cv_splits):
        if f == 0 or f == 1:
            continue
        # load feature caches for fold f
        train_ds = FatherDatasetSubset(ds, train_idx, eval=False)
        test_ds = FatherDatasetSubset(ds, test_idx, eval=True)

        weights_path = os.path.join(
            'weights',
            f'I{"-".join(input_modalities)}_fold{f}.ckpt'
        )

        pl.utilities.seed.seed_everything(seed+f+734890573)
        if do_train:
            trainer = train(f, train_ds, test_ds, input_modalities,
                prefix = prefix + f'_fold{f}' if prefix else None,
                eval_every_epoch=True,
                deterministic=deterministic,
                weights_path=weights_path)
            model = trainer.model
        else:
            model = System.load_from_checkpoint(checkpoint_path=weights_path)

        # ensures that the testing is reproducible regardless of training
        pl.utilities.seed.seed_everything(seed+f+2980374334)
        fold_outputs = test(f, model, test_ds, prefix = prefix + f'_fold{f}' if prefix else None,)
        all_results.append(fold_outputs)
        clear_output(wait=False)

    outputs = [r['proba'].numpy() for r in all_results]
    indices = [r['index'].numpy() for r in all_results]
    metrics = [r['metric'] for r in all_results]
    return metrics, outputs, indices

In [6]:
def do_run(examples, input_modalities, 
    do_train=True, deterministic=True, prefix=''):
    ''' train = True will train the models, and requires 
            model_label_modality = test_label_modality
        train = False will load weights to test the models and does not require
            model_label_modality = test_label_modality
    '''
    print(f'Using {len(examples)} examples')

    # create the feature datasets
    extractors = {}
    if 'video' in input_modalities:
        videos_path = processed_videos_path # os.path.join(processed_videos_path, 'examples')
        extractors['video'] = VideoExtractor(videos_path, transform=get_kinetics_val_transform(8, 256, False))

    if 'accel' in input_modalities:
        accel_ds_path = os.path.join(processed_accel_path, 'subj_accel_interp.pkl')
        extractors['accel'] = AccelExtractor(accel_ds_path)

    ds = FatherDataset(examples, extractors, transform=RemoveConfidence())

    seed = 22    
    metrics, probas, indices = do_cross_validation(
        do_train,
        ds,
        input_modalities=input_modalities,
        deterministic=deterministic,
        seed=seed,
        prefix=f'{prefix}I{"-".join(input_modalities)}')

    torch.cuda.empty_cache()

    return {
        'metrics': metrics,
        'probas': probas,
        'indices': indices,
        'seed': seed
    }

In [7]:
def get_table(do_train=True, deterministic=True):
    examples = pickle.load(open(examples_path, 'rb'))

    all_input_modalities = [
        ('video',),
        # ('poses',),
        # ('accel',),
    ]    

    res = {}
    for input_modalities in all_input_modalities:

        run_results = do_run(
            examples, 
            input_modalities, 
            do_train=do_train,
            deterministic=deterministic)

        res['-'.join(input_modalities)] = run_results
    return res

In [8]:
try:
    res = get_table(do_train=True, deterministic=False)
except Exception:
    print(traceback.format_exc())

In [12]:
res

{'poses': {'metrics': [0.5912555928464226],
  'probas': [array([[-0.88281935, -0.8496961 , -0.816506  , ..., -0.7836585 ,
           -0.7398186 , -0.6024262 ],
          [-0.8199747 , -0.79450303, -0.7811566 , ..., -0.6872438 ,
           -0.6989202 , -0.4826259 ],
          [-0.6740388 , -0.72466534, -0.76161736, ..., -0.26331377,
           -0.266133  , -0.28297275],
          ...,
          [-2.5501676 , -2.4198594 , -2.6573935 , ..., -2.846579  ,
           -2.5305772 , -2.289178  ],
          [-2.2257392 , -2.1153748 , -2.3338811 , ..., -2.743276  ,
           -2.4590673 , -2.2525878 ],
          [-2.4863722 , -2.2748914 , -2.463201  , ..., -3.5208306 ,
           -3.1582515 , -2.9559388 ]], dtype=float32)],
  'indices': [array([  747,   748,   749, ..., 16092, 16093, 16094])],
  'seed': 22}}

In [9]:
res

{'poses': {'metrics': [0.5912555928464226],
  'probas': [array([[-0.88281935, -0.8496961 , -0.816506  , ..., -0.7836585 ,
           -0.7398186 , -0.6024262 ],
          [-0.8199747 , -0.79450303, -0.7811566 , ..., -0.6872438 ,
           -0.6989202 , -0.4826259 ],
          [-0.6740388 , -0.72466534, -0.76161736, ..., -0.26331377,
           -0.266133  , -0.28297275],
          ...,
          [-2.5501676 , -2.4198594 , -2.6573935 , ..., -2.846579  ,
           -2.5305772 , -2.289178  ],
          [-2.2257392 , -2.1153748 , -2.3338811 , ..., -2.743276  ,
           -2.4590673 , -2.2525878 ],
          [-2.4863722 , -2.2748914 , -2.463201  , ..., -3.5208306 ,
           -3.1582515 , -2.9559388 ]], dtype=float32)],
  'indices': [array([  747,   748,   749, ..., 16092, 16093, 16094])],
  'seed': 22}}